# Justice League Stellar Merger History

## Charlotte Christensen, June 11 2025

This jupyter notebook runs Anna Wright's code (/home/christenc/Code/python/AnnaWrite_startrace/RomZoomSHAnalysisScripts) to identify the stars in those halos.

From Anna's Email (June 4, 2025)

I managed to get through the pipeline and remember what everything did well enough to comment it today, but didn't have time to test it on a new halo. However, if you'd like to try it in the next 24 hours or so, I've attached a zip file with steps 1-7 of my pipeline (there are 8 files, but TrackDownStars_rz.ipynb and FixHostIDs_rz.py are really two halves of a single step). Step 0 is creating a tangos db for the simulation you'll be working with and my pipeline uses that and the simulation itself to create an hdf5 file with relevant data for each star particle. The most important bits are a unique host ID for each star particle so that stars that formed in the same halo can be grouped together, even if that halo doesn't exist at z=0, and the orbital circularity of each star particle, which I use to identify members of the stellar halo.
I will be testing it tomorrow on one of the newer Romulus Zooms, so there's a good chance I'll be sending you an updated version very soon with any bug fixes :) 
I apologize for how many pieces the pipeline is in - it really isn't all that complicated, but it was adapted from what I did for the FOGGIE sims and I split the steps of that pipeline up so that I could do as much as possible locally (rather than on Pleiades) and so that I could sanity check as often as possible. Please let me know if you have any issues, questions, or suggestions! I'd definitely be eager to hear what Juan does differently!

In [2]:
import os
os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/Data/Marvel_BN_N10.db'
os.chdir('/home/selvani/MAP/pynbody/AnnaWright_startrace/')

In [3]:
import pynbody as pb
import numpy as np
import pandas as pd
import glob
import h5py
import tangos

# For importing modules
import importlib.util
import sys
from pathlib import Path

In [4]:
# Simulation name and path

# Elena
# simname = 'h329' # not needed?
# res = '100'  # The Near Mint runs not needed?
#res = 'Mint'  # The Mint
simpath = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
basename = 'cptmarvel.cosmo25cmb.4096g5HbwK1BH'

ss_dir = ''#'snapshots_200crit_h329'
sim_base = simpath + ss_dir
ss_z0 = sim_base + basename + '.004096'

outfile_dir = "/home/selvani/MAP/pynbody/stellarhalo_trace_aw/"

### 1) GrabTF_rz.py

Step 1 of stellar halo pipeline

**What it does:**
- Loads the final snapshot (z=0) of the simulation
- Extracts formation time (`tform`) and particle IDs (`iord`) for all star particles that have `tform > 0` (i.e., actual stars, not wind particles)
- Converts formation times to Gyr units
- Creates a 2D array with particle IDs in the first row and formation times in the second row
- Saves this data as a `.npy` file named `<simulation_name>_tf.npy`
- Prints the total number of star particles found as a sanity check

**Input:** Simulation snapshot (specifically the final snapshot `.004096`)

**Output:** `<sim>_tf.npy` - NumPy file containing:
- Row 0: Star particle IDs (`iord`)  
- Row 1: Formation times in Gyr (`tform`)

**Purpose:** This creates the foundation dataset that subsequent steps will use to trace back each star particle to determine which halo it was forming in at its birth time.

* Usage:   `python GrabTF_rz.py <simpath> <output_directory>`
* Example: `python GrabTF_rz.py /path/to/sim/ /path/to/output/`

In [9]:
# Import Anna's code, even if not along my path
file_path = '/home/selvani/MAP/pynbody/AnnaWright_startrace/RomZoomSHAnalysisScripts/GrabTF_rz.py'
module_name = 'GrabTF_rz'

spec = importlib.util.spec_from_file_location(module_name, file_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [10]:
module.main(simpath, outfile_dir)

414596 stars found!
Save to  /home/selvani/MAP/pynbody/stellarhalo_trace_aw/cptmarvel.cosmo25cmb.4096g5HbwK1BH_tf.npy


### 2) LocAtCreation_pool_rz.py

Step 2 of stellar halo pipeline
Identifies the host of each star particle in \<sim\>_tf.npy at the 
time it was formed. **Note that what is stored is NOT the amiga.grp 
ID, but the index of that halo in the tangos database. The amiga.grp
ID can be backed out via tangos with sim[stepnum][halonum].finder_id.** 
(CC, I believe that I edited this so now the halo_)

Output: <sim>_stardata_<snapshot>.h5
        where <snapshot> is the first snapshot that a given process
        analyzed. There will be <nproc> of these files generated
        and processes will not necessarily analyze adjacent snapshots

* Usage:   python LocAtCreation_pool_rz.py <sim> optional:<nproc>
* Example: python LocAtCreation_pool_rz.py r634 2

Includes an optional argument to specify number of processes to run
with; default is 4. Note that this will get reduced if you've specified
more processes than you have snapshots to process.

Note that this has the name of the snapshots directory hardcoded into FindHaloStars.py (L63)
-- Will need to be adjusted
The 

CC: When I did my edits, I moved code into FindHaloStars so that it can be imported for multiprocessing


In [19]:
# Import the module
import os
base_path = '/home/selvani/MAP/pynbody/AnnaWright_startrace'

for root, dirs, files in os.walk(base_path):
    if root not in sys.path:
        print("Adding to sys.path:", root)
        sys.path.append(root)
        
import LocAtCreation_pool_rz

# Set the tangos database
# In terminal
# export TANGOS_DB_CONNECTION=/home/selvani/MAP/Data/Marvel_BN_N10.db
# os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/Data/Marvel_BN_N10.db'

Base path: /home/selvani/MAP/pynbody/AnnaWright_startrace


In [ ]:
import psutil
n_cpus = psutil.cpu_count(logical=False)
LocAtCreation_pool_rz.main(simpath, ss_dir, outfile_dir, n_cpus)

### 3) writeouthosts_rz.py

Step 3 of stellar halo pipeline                                                                                   
For each snapshot, writes out a list of halos that formed stars                                                   
between the last snapshot and this one and the number of stars formed;                                            
as in step 2, note that the IDs of these halos will be their index in                                             
the tangos database, not necessarily their amiga.grp ID. This is used                                             
to construct a unique ID for each star-forming halo in the next step.                                             
                                                                                                                  
Output: <sim>_halostarhosts.txt                                                                                   
                                                                                                                  
Usage:   python writeouthosts_rz.py <sim>                                                                         
Example: python writeouthosts_rz.py r634                                                                          
                                                                                                                  
Note that this is currently set up for MMs, but should be easily adapted                                          
by e.g., changing the paths or adding a path CL argument.    

In [ ]:
# # Import the module
# import os
# base_path = '/home/christenc/Code/python/AnnaWright_startrace'

# for root, dirs, files in os.walk(base_path):
#     if root not in sys.path:
#         sys.path.append(root)
        
import writeouthosts_rz

ModuleNotFoundError: No module named 'writeouthosts_rz'

In [4]:
writeouthosts_rz.main(basename, odir='/home/christenc/Code/Datafiles/stellarhalo_trace_aw/')

#### Extra stuff charlotte added

In [ ]:
print(np.unique(tslist))
print(len(tslist))
print(len(hostlist))

In [ ]:
t = 456
tstr = str(int(t))
tmask = np.where(tslist==t)[0]
np.size(tmask)

### 4) IDUniqueHost_rz.py

Step 4 of stellar halo pipeline                                                                                                                       
Creates a unique ID for each host that forms a star. The format of                                                                                    
this ID is \<last snapshot where host was IDed\>_\<index at this snapshot\>.                                                                              
So, if a host was halo 5 at snapshot 3552 and then merged with halo 1                                                                                 
before the next snapshot, its unique ID will be 3552_5. Stars that form                                                                               
in its main progenitors will also be associated with this ID. These IDs                                                                               
are written out to a file with a similar format to <sim>_halostarhosts.txt.                                                                           
                                                                                                                                                      
Output: <sim>_uniquehalostarhosts.txt                                                                                                                 
                                                                                                                                                      
Usage:   python IDUniqueHost_rz.py <sim>                                                                                                              
Example: python IDUniqueHost_rz.py r634                                                                                                               
                                                                                                                                                      
Note that this is currently set up for MMs, but should be easily adapted                                                                              
by e.g., changing the paths or adding a path CL argument. It is also                                                                                  
designed to accommodate the phantoms that rockstar generates when it                                                                                  
temporarily loses track of a halo, which slows it down quite a bit.                                                                                   
If you're only ever going to be using it with other types of merger                                                                                   
trees, it can be simplified.  

In [7]:
# Import the module
import os
base_path = '/home/christenc/Code/python/AnnaWrite_startrace'

for root, dirs, files in os.walk(base_path):
    if root not in sys.path:
        sys.path.append(root)
        
import IDUniqueHost_rz

# Set the tangos database
# In terminal
# export TANGOS_DB_CONNECTION=/home/christenc/Storage/tangos_db/JL_r200.db
os.environ['TANGOS_DB_CONNECTION'] = '/home/christenc/Storage/tangos_db/JL_r200.db'

In [ ]:

sim = db.get_simulation(cursim+'%')

d = collections.defaultdict(list)

hsfile = odir+cursim+'_halostarhosts.txt'
ofile = odir+cursim+'_uniquehalostarhosts.txt'

main(sim, d, hsfile, ofile)


### 5) StoreUniqueHostID_rz.py

Step 5 of stellar halo pipeline
Stores the unique ID of each star particle's host at formation time.
Creates an hdf5 file that contains this in addition to all of the data
from the <sim>_stardata_<snapshot>.h5 files. Note that all star particles
that don't have a host in the snapshot after they formed will be assigned 
a unique ID of <snapshot_index>_0 and a particle host (i.e., host at
formation time) of -1. It is recommended that you use the TrackDownStars
Jupyter notebook to try to manually identify hosts for these stars and then
use FixHostIDs_rz.py to amend <sim>_allhalostardata.h5.

Output: <sim>_allhalostardata.h5

Usage:   python StoreUniqueHostID_rz.py <sim>
Example: python StoreUniqueHostID_rz.py r634 

Note that this is currently set up for MMs, but should be easily adapted 
by e.g., changing the paths or adding a path CL argument.



### 6a) TrackDownStars_rz.ipnb

### 6b) FixHostID_rz

Optional: Step 6b of stellar halo pipeline
Updates the host_ID values stored in the allhalostardata hdf5 file based
on user input. This is designed as a follow-up to TrackDownStars and can 
be used in a couple of ways. If ffile=True, this script will look for 
numpy files with names <sim>_new_ID_?.npy and will assign the particles
with the iords in a given file to new_ID. If ffile=False, it will assign
all particles with a host_ID in the old_ID list to new_ID.

Output: <sim>_allhalostardata_upd.h5

Usage:   python FixHostIDs_rz.py <sim>
Example: python FixHostIDs_rz.py r634 

The script will print out all host_IDs for which the number of assigned particles 
changed and how many particles each gained/lost. If the output looks correct,
the user should manually rename <sim>_allhalostardata_upd.h5 to <sim>_allhalostardata.h5.
It's often necessary to go back and forth between this and TrackDownStars, in which 
case I usually move the *.npy files that have already been processed to a subfolder. 


### 6c) CompTwoHalos_rz

Optional: Step 6c of stellar halo pipeline
Compares two halos to see how likely it is that one is
the main progenitor of the other based on how many particles
they have in common. This is particularly useful when you've
used a merger tree constructor that doesn't use phantoms or
some equivalent and may therefore fail to connect a halo at
snapshot1 to the same halo at snapshot3 if it lost track of it
at snapshot2. This information can then be used with FixHostIDs_rz
to merge two unique IDs and/or to create a new link in the 
relevant tangos db.

Usage: python CompTwoHalos_rz.py <sim> <halo1> <halo2>
Example: python CompTwoHalos_rz.py r718 0136_4 0192_3

Output: prints out the fraction of <halo1>'s DM particles
that are in <halo2> and vice-versa.

It takes three arguments: the simulation you're working with
and the tangos IDs of the two halos you want to compare, which
are formatted as <snapshot>_<IDatsnapshot>. Note that this ID
is assumed to be the tangos ID, not necessarily the amiga.grp
ID.
